Это бонусная необязательная тема, необходим доступ к микрофону.

# Тема 22b. Клонирование голоса

Имея возможность создавать векторные представления для речи, можно проводить с ними различные операции, например, клонировать голос, т.е. заставить синтетический голос говорить похожим на вас образом.

Здесь посмотрим только пример [1]:

![img](https://miro.medium.com/max/7356/1*GIcVhhSAu3VWBgJGSxdmUA.png)

- Имеется набор предложений в текстовом виде и соответствующие зачитывания этих предложений дикторами в виде мел-спектрограмм.

- Звуковой кодер создает векторное представление для спектрограмм. 

- Текстовый кодер создает векторные представления для предложений.

- Эти два векторных представления объединяются в одно.

- Из такого представления генерируется мел-спектрограмма (при помощи модуля внимания и декодера)

- Сгенерированная спектрограмма сравнивается с исходной и задачей обучения является сделать так, чтобы они совпадали как можно точнее.

- после обучения такую спектрограмму используют для генерации звука.




Такая реализация есть в библиотеке [CorentinJ/Real-Time-Voice-Cloning](https://github.com/CorentinJ/Real-Time-Voice-Cloning), пример с ее использованием [2].




In [ ]:
#@title Установка CorentinJ/Real-Time-Voice-Cloning

#@markdown * клонируем проект
#@markdown * загружаем обученные модели
#@markdown * инициализируем модели

%tensorflow_version 1.x
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CorentinJ/Real-Time-Voice-Cloning.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install
  !git clone -q --recursive {git_repo_url}
  # install dependencies
  !cd {project_name} && pip install -q -r requirements.txt
  !pip install -q gdown
  !apt-get install -qq libportaudio2
  !pip install -q https://github.com/tugstugi/dl-colab-notebooks/archive/colab_utils.zip

  # download pretrained model
  !cd {project_name} && wget https://github.com/blue-fish/Real-Time-Voice-Cloning/releases/download/v1.0/pretrained.zip && unzip -o pretrained.zip

import sys
sys.path.append(project_name)

from IPython.display import display, Audio, clear_output
from IPython.utils import io
import ipywidgets as widgets
import numpy as np
from dl_colab_notebooks.audio import record_audio, upload_audio

from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path

encoder.load_model(project_name / Path("encoder/saved_models/pretrained.pt"))
synthesizer = Synthesizer(project_name / Path("synthesizer/saved_models/pretrained/pretrained.pt"))
vocoder.load_model(project_name / Path("vocoder/saved_models/pretrained/pretrained.pt"))

TensorFlow 1.x selected.
     |████████████████████████████████| 81kB 3.6MB/s 
     |████████████████████████████████| 686kB 6.8MB/s 
     |████████████████████████████████| 10.3MB 8.6MB/s 
     |████████████████████████████████| 14.5MB 205kB/s 
     |████████████████████████████████| 245kB 37.9MB/s 
     |████████████████████████████████| 8.3MB 32.0MB/s 
     |████████████████████████████████| 71kB 8.4MB/s 
     |████████████████████████████████| 1.2MB 33.6MB/s 
     |████████████████████████████████| 71kB 8.4MB/s 
     |████████████████████████████████| 59.9MB 127kB/s 
     |████████████████████████████████| 317kB 36.8MB/s 
ERROR: tensorflow 1.15.2 has requirement gast==0.2.2, but you'll have gast 0.4.0 which is incompatible.
ERROR: lucid 0.3.10 has requirement numpy<=1.19, but you'll have numpy 1.19.4 which is incompatible.
ERROR: kapre 0.3.5 has requirement tensorflow>=2.0.0, but you'll have tensorflow 1.15.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==

In [ ]:
#@title Запись или Загрузка речи
#@markdown * Или запишите речь с микрофона, или загрузите из файла (.mp3 or .wav) 

SAMPLE_RATE = 22050
record_or_upload = "Record" #@param ["Record", "Upload (.mp3 or .wav)"]
record_seconds =   10#@param {type:"number", min:1, max:10, step:1}

embedding = None
def _compute_embedding(audio):
  display(Audio(audio, rate=SAMPLE_RATE, autoplay=True))
  global embedding
  embedding = None
  embedding = encoder.embed_utterance(encoder.preprocess_wav(audio, SAMPLE_RATE))
def _record_audio(b):
  clear_output()
  audio = record_audio(record_seconds, sample_rate=SAMPLE_RATE)
  _compute_embedding(audio)
def _upload_audio(b):
  clear_output()
  audio = upload_audio(sample_rate=SAMPLE_RATE)
  _compute_embedding(audio)

if record_or_upload == "Record":
  button = widgets.Button(description="Record Your Voice")
  button.on_click(_record_audio)
  display(button)
else:
  #button = widgets.Button(description="Upload Voice File")
  #button.on_click(_upload_audio)
  _upload_audio("")

Starting recording for 10 seconds...


<IPython.core.display.Javascript object>

Finished recording!


In [ ]:
#@title Синтез речи заданным голосом { run: "auto" }
text = "One of the two people who tested positive for the novel coronavirus in the United Kingdom is a student at the University of York in northern England." #@param {type:"string"}
  
def synthesize(embed, text):
  print("Synthesizing new audio...")
  #with io.capture_output() as captured:
  specs = synthesizer.synthesize_spectrograms([text], [embed])
  generated_wav = vocoder.infer_waveform(specs[0])
  generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
  clear_output()
  display(Audio(generated_wav, rate=synthesizer.sample_rate, autoplay=True))

if embedding is None:
  print("first record a voice or upload a voice file!")
else:
  synthesize(embedding, text)

Just for fun

# Ссылки

Использованы и адаптированы материалы:

1. https://medium.com/analytics-vidhya/the-intuition-behind-voice-cloning-with-5-seconds-of-audio-5989e9b2e042

2. https://github.com/tugstugi/dl-colab-notebooks/blob/master/notebooks/RealTimeVoiceCloning.ipynb